# AI基础设施矩阵 - K8s GPU作业集成

这个Notebook提供了与Kubernetes GPU集群集成的完整功能。

## 功能特性
- 🚀 一键提交Python脚本到K8s GPU集群
- 📊 实时GPU资源监控
- 🔄 作业状态跟踪和日志查看
- 💾 NFS共享存储集成
- 🛠️ 智能资源分配和调度

**使用前请先运行下面的初始化单元格！**

In [ ]:
# 初始化环境和依赖
import sys
import os
import subprocess
import importlib.util

def install_package(package):
    """安装Python包"""
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

def check_and_install(packages):
    """检查并安装必要的包"""
    for package in packages:
        package_name = package.split('==')[0]  # 处理版本号
        spec = importlib.util.find_spec(package_name.replace('-', '_'))
        if spec is None:
            print(f"正在安装 {package}...")
            install_package(package)
        else:
            print(f"✅ {package_name} 已安装")

# 安装必要的依赖
required_packages = [
    'requests>=2.25.0',
    'ipywidgets>=7.6.0',
    'pandas>=1.3.0',
    'matplotlib>=3.4.0',
    'pyyaml>=6.0'
]

print("🔧 初始化AI基础设施矩阵K8s GPU集成环境...")
check_and_install(required_packages)

# 设置环境变量
os.environ['AI_INFRA_API_URL'] = os.environ.get('AI_INFRA_API_URL', 'http://localhost:8080')
os.environ['JUPYTERHUB_K8S_NAMESPACE'] = os.environ.get('JUPYTERHUB_K8S_NAMESPACE', 'jupyterhub-jobs')

print("✅ 环境初始化完成！")
print(f"📡 API地址: {os.environ['AI_INFRA_API_URL']}")
print(f"🎯 K8s命名空间: {os.environ['JUPYTERHUB_K8S_NAMESPACE']}")

In [ ]:
# K8s GPU集成客户端类
import requests
import json
import time
from datetime import datetime
from typing import Dict, List, Optional
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

class AIInfraK8sClient:
    """AI基础设施矩阵K8s GPU集成客户端"""
    
    def __init__(self, api_url: str = None):
        self.api_url = api_url or os.environ.get('AI_INFRA_API_URL', 'http://localhost:8080')
        self.api_base = f"{self.api_url}/api/v1/jupyterhub"
        self.session = requests.Session()
    
    def get_gpu_status(self) -> Dict:
        """获取GPU资源状态"""
        try:
            response = self.session.get(f"{self.api_base}/gpu/status")
            response.raise_for_status()
            return response.json()
        except Exception as e:
            return {"error": str(e), "total_gpus": 0, "available_gpus": 0}
    
    def find_gpu_nodes(self, gpu_count: int = 1, gpu_type: str = "") -> Dict:
        """查找适合的GPU节点"""
        params = {"gpu_count": gpu_count}
        if gpu_type:
            params["gpu_type"] = gpu_type
        
        try:
            response = self.session.get(f"{self.api_base}/gpu/nodes", params=params)
            response.raise_for_status()
            return response.json()
        except Exception as e:
            return {"error": str(e), "suitable_nodes": [], "count": 0}
    
    def submit_job(self, name: str, script: str, **kwargs) -> Dict:
        """提交Python脚本作业"""
        data = {
            "name": name,
            "script": script,
            "requirements": kwargs.get("requirements", []),
            "gpu_required": kwargs.get("gpu_required", False),
            "gpu_count": kwargs.get("gpu_count", 1),
            "gpu_type": kwargs.get("gpu_type", ""),
            "memory_mb": kwargs.get("memory_mb", 1024),
            "cpu_cores": kwargs.get("cpu_cores", 1),
            "environment": kwargs.get("environment", {}),
            "working_dir": kwargs.get("working_dir", "/workspace"),
            "output_path": kwargs.get("output_path", "/shared/output")
        }
        
        try:
            response = self.session.post(f"{self.api_base}/jobs/submit", json=data)
            response.raise_for_status()
            return response.json()
        except Exception as e:
            return {"error": str(e)}
    
    def get_job_status(self, job_name: str) -> Dict:
        """获取作业状态"""
        try:
            response = self.session.get(f"{self.api_base}/jobs/{job_name}/status")
            response.raise_for_status()
            return response.json()
        except Exception as e:
            return {"error": str(e), "status": "unknown"}
    
    def wait_for_completion(self, job_name: str, timeout: int = 3600, callback=None):
        """等待作业完成"""
        start_time = time.time()
        
        while time.time() - start_time < timeout:
            status = self.get_job_status(job_name)
            
            if callback:
                callback(status)
            
            if status.get("status") in ["completed", "failed"]:
                return status
            
            time.sleep(5)
        
        return {"error": "Timeout", "status": "timeout"}

# 创建全局客户端实例
ai_client = AIInfraK8sClient()
print("🤖 AI基础设施K8s客户端已初始化")
print(f"🔗 连接到: {ai_client.api_url}")

In [ ]:
# GPU资源监控面板
def create_gpu_monitor():
    """创建GPU资源监控面板"""
    
    status_output = widgets.Output()
    refresh_btn = widgets.Button(description="🔄 刷新", button_style='info')
    auto_refresh = widgets.Checkbox(value=False, description="自动刷新(10s)")
    
    def update_status():
        with status_output:
            clear_output(wait=True)
            status = ai_client.get_gpu_status()
            
            if "error" in status:
                display(HTML(f"<div style='color: red;'>❌ 连接错误: {status['error']}</div>"))
                return
            
            # 创建状态显示
            html = f"""
            <div style='border: 1px solid #ddd; padding: 15px; border-radius: 5px; background: #f9f9f9;'>
                <h3>🎯 GPU集群资源状态</h3>
                <div style='margin: 10px 0;'>
                    <span style='font-weight: bold; color: #2e7d32;'>总GPU数量:</span> {status.get('total_gpus', 0)}<br>
                    <span style='font-weight: bold; color: #1976d2;'>可用GPU:</span> {status.get('available_gpus', 0)}<br>
                    <span style='font-weight: bold; color: #d32f2f;'>已用GPU:</span> {status.get('used_gpus', 0)}<br>
                    <span style='font-weight: bold;'>更新时间:</span> {status.get('last_updated', 'N/A')}
                </div>
            """
            
            # GPU节点详情
            if 'gpu_nodes' in status and status['gpu_nodes']:
                html += "<h4>📋 GPU节点详情:</h4><table style='width: 100%; border-collapse: collapse;'>"
                html += "<tr style='background: #e3f2fd;'><th>节点名称</th><th>GPU类型</th><th>总数</th><th>可用</th><th>状态</th></tr>"
                
                for node in status['gpu_nodes']:
                    status_icon = "✅" if node.get('schedulable') else "❌"
                    html += f"""
                    <tr style='border-bottom: 1px solid #ddd;'>
                        <td style='padding: 5px;'>{node.get('node_name', 'N/A')}</td>
                        <td style='padding: 5px;'>{node.get('gpu_type', 'N/A')}</td>
                        <td style='padding: 5px;'>{node.get('gpu_count', 0)}</td>
                        <td style='padding: 5px;'>{node.get('available_gpus', 0)}</td>
                        <td style='padding: 5px;'>{status_icon}</td>
                    </tr>
                    """
                html += "</table>"
            else:
                html += "<p style='color: orange;'>⚠️ 未找到GPU节点或节点信息不可用</p>"
            
            html += "</div>"
            display(HTML(html))
    
    def on_refresh_click(b):
        update_status()
    
    refresh_btn.on_click(on_refresh_click)
    
    # 自动刷新逻辑
    import threading
    def auto_refresh_worker():
        while auto_refresh.value:
            update_status()
            time.sleep(10)
    
    def on_auto_refresh_change(change):
        if change['new']:
            thread = threading.Thread(target=auto_refresh_worker, daemon=True)
            thread.start()
    
    auto_refresh.observe(on_auto_refresh_change, names='value')
    
    # 初始加载
    update_status()
    
    return widgets.VBox([
        widgets.HBox([refresh_btn, auto_refresh]),
        status_output
    ])

# 显示GPU监控面板
print("📊 创建GPU资源监控面板...")
gpu_monitor = create_gpu_monitor()
display(gpu_monitor)

In [ ]:
# 作业提交界面
def create_job_submission_panel():
    """创建作业提交面板"""
    
    # 表单控件
    job_name = widgets.Text(placeholder="输入作业名称", description="作业名称:")
    script_area = widgets.Textarea(
        placeholder="输入Python脚本代码...",
        description="Python脚本:",
        rows=10,
        layout=widgets.Layout(width='100%')
    )
    
    requirements = widgets.Text(
        placeholder="例如: torch pandas numpy",
        description="依赖包:"
    )
    
    gpu_required = widgets.Checkbox(value=False, description="需要GPU")
    gpu_count = widgets.IntSlider(value=1, min=1, max=8, description="GPU数量:")
    gpu_type = widgets.Text(placeholder="例如: rtx4090", description="GPU类型:")
    
    memory_mb = widgets.IntSlider(value=2048, min=512, max=16384, step=512, description="内存(MB):")
    cpu_cores = widgets.IntSlider(value=2, min=1, max=16, description="CPU核心:")
    
    submit_btn = widgets.Button(description="🚀 提交作业", button_style='success')
    output_area = widgets.Output()
    
    # 预设示例
    examples = widgets.Dropdown(
        options=[
            ('选择示例...', ''),
            ('GPU检测脚本', 'gpu_test'),
            ('数据分析脚本', 'data_analysis'),
            ('机器学习训练', 'ml_training')
        ],
        description="示例:"
    )
    
    def load_example(change):
        if change['new'] == 'gpu_test':
            job_name.value = "GPU检测任务"
            script_area.value = """
import torch
import time
from datetime import datetime

print(f"开始GPU检测 - {datetime.now()}")
print(f"PyTorch版本: {torch.__version__}")
print(f"CUDA可用: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    device_count = torch.cuda.device_count()
    print(f"GPU设备数量: {device_count}")
    
    for i in range(device_count):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
        props = torch.cuda.get_device_properties(i)
        print(f"  内存: {props.total_memory / 1e9:.1f} GB")
    
    # 简单GPU计算测试
    print("\n执行GPU矩阵计算测试...")
    device = torch.device('cuda')
    a = torch.randn(1000, 1000, device=device)
    b = torch.randn(1000, 1000, device=device)
    
    start_time = time.time()
    c = torch.mm(a, b)
    compute_time = time.time() - start_time
    
    print(f"矩阵乘法耗时: {compute_time:.4f}秒")
    print(f"结果检查: mean={c.mean().item():.4f}")
else:
    print("GPU不可用，使用CPU计算")

print(f"任务完成 - {datetime.now()}")
"""
            requirements.value = "torch"
            gpu_required.value = True
            memory_mb.value = 4096
            
        elif change['new'] == 'data_analysis':
            job_name.value = "数据分析任务"
            script_area.value = """
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import json

print(f"开始数据分析 - {datetime.now()}")

# 生成示例数据
np.random.seed(42)
data = {
    'A': np.random.randn(1000),
    'B': np.random.randn(1000) * 2 + 1,
    'C': np.random.choice(['X', 'Y', 'Z'], 1000),
    'D': np.random.uniform(0, 100, 1000)
}

df = pd.DataFrame(data)
print(f"数据形状: {df.shape}")
print("\n数据统计:")
print(df.describe())

# 分组分析
print("\n按类别分组分析:")
grouped = df.groupby('C').agg({
    'A': ['mean', 'std'],
    'B': ['mean', 'std'],
    'D': ['mean', 'max', 'min']
})
print(grouped)

# 保存结果
results = {
    'timestamp': datetime.now().isoformat(),
    'data_shape': df.shape,
    'summary_stats': df.describe().to_dict(),
    'category_analysis': grouped.to_dict()
}

with open('/shared/data_analysis_results.json', 'w') as f:
    json.dump(results, f, indent=2)

print("\n结果已保存到 /shared/data_analysis_results.json")
print(f"分析完成 - {datetime.now()}")
"""
            requirements.value = "pandas numpy matplotlib"
            gpu_required.value = False
            memory_mb.value = 2048
            
        elif change['new'] == 'ml_training':
            job_name.value = "机器学习训练"
            script_area.value = """
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from datetime import datetime
import json

print(f"开始机器学习训练 - {datetime.now()}")

# 设置设备
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"使用设备: {device}")

# 生成示例数据
torch.manual_seed(42)
X = torch.randn(1000, 10, device=device)
y = torch.sum(X[:, :5], dim=1) + torch.randn(1000, device=device) * 0.1

# 定义简单神经网络
class SimpleNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(10, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )
    
    def forward(self, x):
        return self.layers(x).squeeze()

# 训练模型
model = SimpleNet().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

print("开始训练...")
losses = []

for epoch in range(100):
    optimizer.zero_grad()
    outputs = model(X)
    loss = criterion(outputs, y)
    loss.backward()
    optimizer.step()
    
    losses.append(loss.item())
    
    if (epoch + 1) % 20 == 0:
        print(f"Epoch [{epoch+1}/100], Loss: {loss.item():.6f}")

# 保存结果
results = {
    'timestamp': datetime.now().isoformat(),
    'device': str(device),
    'final_loss': losses[-1],
    'training_losses': losses,
    'model_parameters': sum(p.numel() for p in model.parameters())
}

with open('/shared/ml_training_results.json', 'w') as f:
    json.dump(results, f, indent=2)

print(f"\n训练完成 - 最终损失: {losses[-1]:.6f}")
print("结果已保存到 /shared/ml_training_results.json")
print(f"训练结束 - {datetime.now()}")
"""
            requirements.value = "torch"
            gpu_required.value = True
            memory_mb.value = 4096
    
    examples.observe(load_example, names='value')
    
    def on_submit_click(b):
        with output_area:
            clear_output(wait=True)
            
            if not job_name.value or not script_area.value:
                display(HTML("<div style='color: red;'>❌ 请填写作业名称和脚本内容</div>"))
                return
            
            print("📤 正在提交作业...")
            
            # 准备作业参数
            job_params = {
                "requirements": [pkg.strip() for pkg in requirements.value.split() if pkg.strip()],
                "gpu_required": gpu_required.value,
                "gpu_count": gpu_count.value if gpu_required.value else 0,
                "gpu_type": gpu_type.value,
                "memory_mb": memory_mb.value,
                "cpu_cores": cpu_cores.value
            }
            
            # 提交作业
            result = ai_client.submit_job(job_name.value, script_area.value, **job_params)
            
            if "error" in result:
                display(HTML(f"<div style='color: red;'>❌ 提交失败: {result['error']}</div>"))
                return
            
            job_id = result.get('job_id')
            job_name_submitted = result.get('job_name')
            
            display(HTML(f"""
            <div style='border: 1px solid #4caf50; padding: 15px; border-radius: 5px; background: #e8f5e8;'>
                <h3>✅ 作业提交成功!</h3>
                <p><strong>作业ID:</strong> {job_id}</p>
                <p><strong>作业名称:</strong> {job_name_submitted}</p>
                <p><strong>状态:</strong> {result.get('status', 'submitted')}</p>
            </div>
            """))
            
            # 开始监控作业状态
            print("\n🔍 监控作业状态...")
            
            def status_callback(status):
                current_status = status.get('status', 'unknown')
                print(f"⏰ {datetime.now().strftime('%H:%M:%S')} - 状态: {current_status}")
            
            final_status = ai_client.wait_for_completion(job_name_submitted, callback=status_callback)
            
            if final_status.get('status') == 'completed':
                display(HTML("<div style='color: green; font-weight: bold;'>🎉 作业执行完成!</div>"))
            elif final_status.get('status') == 'failed':
                display(HTML(f"<div style='color: red;'>❌ 作业执行失败: {final_status.get('error_message', '未知错误')}</div>"))
            else:
                display(HTML(f"<div style='color: orange;'>⏳ 作业状态: {final_status.get('status', 'unknown')}</div>"))
    
    submit_btn.on_click(on_submit_click)
    
    # 布局
    form_layout = widgets.VBox([
        widgets.HTML("<h3>🚀 K8s GPU作业提交</h3>"),
        examples,
        job_name,
        script_area,
        requirements,
        widgets.HBox([gpu_required, gpu_count, gpu_type]),
        widgets.HBox([memory_mb, cpu_cores]),
        submit_btn,
        output_area
    ])
    
    return form_layout

# 显示作业提交面板
print("📝 创建作业提交面板...")
job_panel = create_job_submission_panel()
display(job_panel)

In [ ]:
# 快速操作工具栏
def create_quick_actions():
    """创建快速操作工具栏"""
    
    health_btn = widgets.Button(description="🩺 健康检查", button_style='info')
    gpu_test_btn = widgets.Button(description="🔬 GPU测试", button_style='warning')
    examples_btn = widgets.Button(description="📚 查看示例", button_style='primary')
    
    output = widgets.Output()
    
    def health_check(b):
        with output:
            clear_output(wait=True)
            print("🩺 执行健康检查...")
            
            try:
                response = ai_client.session.get(f"{ai_client.api_url}/api/v1/jupyterhub/health")
                if response.status_code == 200:
                    print("✅ API服务正常")
                else:
                    print(f"❌ API服务异常: {response.status_code}")
            except Exception as e:
                print(f"❌ 连接失败: {e}")
            
            # 检查GPU状态
            gpu_status = ai_client.get_gpu_status()
            if "error" not in gpu_status:
                print(f"✅ GPU集群连接正常 - 可用GPU: {gpu_status.get('available_gpus', 0)}")
            else:
                print(f"❌ GPU集群连接异常: {gpu_status['error']}")
    
    def quick_gpu_test(b):
        with output:
            clear_output(wait=True)
            print("🔬 启动GPU快速测试...")
            
            test_script = """
import torch
print(f"PyTorch版本: {torch.__version__}")
print(f"CUDA可用: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU数量: {torch.cuda.device_count()}")
    for i in range(torch.cuda.device_count()):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
print("GPU测试完成")
"""
            
            result = ai_client.submit_job(
                "GPU快速测试",
                test_script,
                requirements=["torch"],
                gpu_required=True,
                memory_mb=1024
            )
            
            if "error" in result:
                print(f"❌ 测试提交失败: {result['error']}")
            else:
                print(f"✅ 测试作业已提交: {result.get('job_name')}")
                print("请等待几分钟查看结果...")
    
    def show_examples(b):
        with output:
            clear_output(wait=True)
            display(HTML("""
            <div style='border: 1px solid #ddd; padding: 15px; border-radius: 5px;'>
                <h3>📚 使用示例</h3>
                
                <h4>1. GPU检测脚本</h4>
                <p>检测GPU设备信息和执行简单的矩阵计算测试</p>
                <ul>
                    <li>需要GPU: ✅</li>
                    <li>依赖: torch</li>
                    <li>内存: 4GB</li>
                </ul>
                
                <h4>2. 数据分析脚本</h4>
                <p>使用pandas和numpy进行数据处理和统计分析</p>
                <ul>
                    <li>需要GPU: ❌</li>
                    <li>依赖: pandas numpy matplotlib</li>
                    <li>内存: 2GB</li>
                </ul>
                
                <h4>3. 机器学习训练</h4>
                <p>使用PyTorch训练简单的神经网络模型</p>
                <ul>
                    <li>需要GPU: ✅</li>
                    <li>依赖: torch</li>
                    <li>内存: 4GB</li>
                </ul>
                
                <h4>💡 使用提示</h4>
                <ul>
                    <li>在作业提交面板选择相应示例自动填充代码</li>
                    <li>可以根据需要修改脚本内容和资源配置</li>
                    <li>作业结果会保存到/shared目录下</li>
                    <li>使用GPU监控面板查看集群资源状态</li>
                </ul>
            </div>
            """))
    
    health_btn.on_click(health_check)
    gpu_test_btn.on_click(quick_gpu_test)
    examples_btn.on_click(show_examples)
    
    return widgets.VBox([
        widgets.HTML("<h3>⚡ 快速操作</h3>"),
        widgets.HBox([health_btn, gpu_test_btn, examples_btn]),
        output
    ])

# 显示快速操作面板
print("⚡ 创建快速操作工具栏...")
quick_actions = create_quick_actions()
display(quick_actions)

## 🎯 使用指南

### 1. 监控GPU资源
- 使用上面的GPU监控面板查看集群状态
- 可以开启自动刷新功能
- 查看各个GPU节点的详细信息

### 2. 提交作业
- 在作业提交面板填写作业信息
- 可以选择预设示例快速开始
- 根据需要配置GPU和内存资源
- 提交后自动监控作业状态

### 3. 快速操作
- 健康检查: 验证API服务和GPU集群连接
- GPU测试: 快速提交GPU检测作业
- 查看示例: 了解不同类型的作业示例

### 4. 结果查看
- 作业结果保存在`/shared`目录下
- 可以通过文件浏览器查看输出文件
- JSON格式的结果可以直接加载分析

---

**📝 注意事项:**
- 确保后端API服务正在运行
- GPU作业需要集群有可用的GPU节点
- 大型作业请合理配置内存和CPU资源
- 可以通过kubectl命令查看K8s集群中的作业状态

**🔗 相关资源:**
- API文档: http://localhost:8080/swagger/
- GPU监控: 使用上面的监控面板
- 作业日志: `kubectl logs -f job/<job-name> -n jupyterhub-jobs`